### DATA PRE-PROCESSING

In [1]:
import pandas as pd    # pandas library for data frame handling
import re              # regular expression library to extract relevant data
from datetime import * # to convert date into the required format

In [2]:
#reading the raw file 
path_to_data = 'UofS_access_log' # enter the name of your file
raw_data_SHAARANG = pd.read_csv(filepath_or_buffer = path_to_data, sep = '\n', encoding="ISO-8859-1", header=None, names=['logs'])

In [20]:
raw_data_SHAARANG.head(4)

,logs
0,202.32.92.47 - - [01/Jun/1995:00:00:59 -0600] ...
1,ix-or7-27.ix.netcom.com - - [01/Jun/1995:00:02...
2,ram0.huji.ac.il - - [01/Jun/1995:00:05:44 -060...
3,eagle40.sasknet.sk.ca - - [01/Jun/1995:00:08:0...


#### the raw file is comprised of only non-null object, as best practice, we will still attempt to detect any anomaly in the data such as NAN or NULL

In [4]:
raw_data_SHAARANG.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2408625 entries, 0 to 2408624
Data columns (total 1 columns):
logs    object
dtypes: object(1)
memory usage: 18.4+ MB


##### Regular Exressions to extract the required data : host, date, status and bytes

In [5]:
regex_Host = r"(^\S+)\s"
regex_datetime = r"\[(.*)\]"
regex_status = r"\s(\d{3})\s"
regex_bytes = r"\s(\d+)$"

### For each record extracting the pertinent Host name in an empty list

In [8]:
Hosts_Shaarang = [re.search(regex_Host, record).group(1)
               if re.search(regex_Host, record)
               else "No match found"
               for record in raw_data_SHAARANG["logs"]]
Hosts_Shaarang[0:3]

['202.32.92.47', 'ix-or7-27.ix.netcom.com', 'ram0.huji.ac.il']

In [9]:
print(len(Hosts_Shaarang))

2408625


### For each record extracting the pertinent date in an empty list

In [43]:
date_list = [re.search(regex_datetime, record).group(1)
               if re.search(regex_datetime, record)
               else "No match found"
               for record in raw_data_SHAARANG["logs"]]

date_list[0:3]

['01/Jun/1995:00:00:59 -0600',
 '01/Jun/1995:00:02:51 -0600',
 '01/Jun/1995:00:05:44 -0600']

### Extracting the Status Code

In [13]:
status_Shaarang = [re.search(regex_status, record).group(1) 
               if re.search(regex_status, record)
               else "No match found"
               for record in raw_data_SHAARANG["logs"]]
status_Shaarang[0:3]

['200', '200', '200']

### Extracting the Bytes from each record

In [16]:
bytes_Shaarang = [re.search(regex_bytes, record).group(1) # this command retrieves your first match
               if re.search(regex_bytes, record)
               else "No match found"
               for record in raw_data_SHAARANG["logs"]]
bytes_Shaarang[0:3]

['271', '205908', '271']

### Checking the length of all the Lists prepared above

In [44]:
len(bytes_Shaarang)==len(status_Shaarang)==len(date_list)==len(Hosts_Shaarang) 
# if lengths are same then we can go ahead and join them

True

### Coverting the lists into series and Concatinating them to obtain a final data frame

In [46]:
status_Shaarang=pd.Series(status_Shaarang)
bytes_Shaarang=pd.Series(bytes_Shaarang)
date_Shaarang=pd.Series(date_list)
Hosts_Shaarang=pd.Series(Hosts_Shaarang)

In [47]:
clean_data_Shaarang=pd.concat([Hosts_Shaarang,date_Shaarang,status_Shaarang,bytes_Shaarang], axis=1)

In [48]:
clean_data_Shaarang.columns=["Hosts","Date","Status","Bytes"]

In [49]:
clean_data_Shaarang.head()

,Hosts,Date,Status,Bytes
0,202.32.92.47,01/Jun/1995:00:00:59 -0600,200,271
1,ix-or7-27.ix.netcom.com,01/Jun/1995:00:02:51 -0600,200,205908
2,ram0.huji.ac.il,01/Jun/1995:00:05:44 -0600,200,271
3,eagle40.sasknet.sk.ca,01/Jun/1995:00:08:06 -0600,200,1116
4,eagle40.sasknet.sk.ca,01/Jun/1995:00:08:19 -0600,200,49649


### Data Sanity Check

In [50]:
clean_data_Shaarang.Hosts.isnull().sum()

0

In [51]:
clean_data_Shaarang.Bytes.isnull().sum()

0

In [52]:
clean_data_Shaarang.Status.isnull().sum()

0

In [53]:
clean_data_Shaarang.Date.isnull().sum()

0

## Clean Data Excel Source for Tableau Analysis

In [54]:
clean_data_Shaarang.to_excel(r'ShaarangCleanDataMidterm.xlsx', index = False)

##   Number of Records in the final DataSet

In [55]:
len(clean_data_Shaarang)

2408625

##  Timestamps for oldest and most recent record

In [56]:
clean_data_Shaarang.Date.agg(['min','max'])

min    01/Aug/1995:00:00:05 -0600
max    31/Oct/1995:23:59:55 -0600
Name: Date, dtype: object